# Various LinkedIn authentication attempts

As of this writing, none of these approaches succeeds in authenticating to the LinkedIn Contacts API

This was my first attempt to understand the cookie-based authentication used by most LinkedIn pages.  This allowed me to discover the lidc cookie:

In [ ]:
import requests
email = 'REPLACE_ME'
password = 'REPLACE_ME'

In [ ]:
r = requests.get('https://www.linkedin.com', auth=(email, password))

In [ ]:
r.status_code

In [ ]:
r.cookies['lidc']

NOTES:


Cookie that results in 200 status:
"b=OB69:g=61:u=183:i=1482278326:t=1482287530:s=AQHDTpXv6CLdihag8_dlKDlMdVt5mQBj"	

Cookie that results in 401 status:
"b=OGST00:g=43:u=1:i=1482278770:t=1482365170:s=AQED7FbwWR3PM4pp2jhve3wCmI9lQXmG"

Cookie that results in 200 status:
"b=OB69:g=61:u=183:i=1482278884:t=1482287530:s=AQEP_-YzK7aGOGTQIHom7i28j6aeREnl"

Here's one attempt to brute-force my way into the Contacts API:

In [ ]:
url = 'https://www.linkedin.com/connected/api/v2/contacts?start=40&count=10&fields=id%2Cname%2CfirstName%2ClastName%2Ccompany%2Ctitle%2Clocation%2Ctags%2Cemails%2Csources%2CdisplaySources%2CconnectionDate%2CsecureProfileImageUrl&sort=CREATED_DESC&_=1481999304007'
cookies = dict(lidc="b=OB69:g=61:u=183:i=1482278884:t=1482287530:s=AQEP_-YzK7aGOGTQIHom7i28j6aeREnl")
r2 = requests.get(url, cookies=cookies)

In [ ]:
r2.status_code

Here's another approach bastardized from this article:
http://stackoverflow.com/questions/25214959/cookie-authentication-with-python-requests#25218910

and this Requests documentation on the Session object: 
http://docs.python-requests.org/en/master/user/advanced/

In [ ]:
url1 = 'https://www.linkedin.com'
url2 = 'https://www.linkedin.com/connected/api/v2/contacts?start=40&count=10&fields=id%2Cname%2CfirstName%2ClastName%2Ccompany%2Ctitle%2Clocation%2Ctags%2Cemails%2Csources%2CdisplaySources%2CconnectionDate%2CsecureProfileImageUrl&sort=CREATED_DESC&_=1481999304007'
s = requests.Session()
s.get(url1, auth=(email, password))
print(s.cookies)

In [ ]:
s.get(url2)

In [ ]:
s.status_code

Here's another approach bastardized from this article:
http://stackoverflow.com/questions/18907503/logging-in-to-linkedin-with-python-requests-sessions#18908282

In [ ]:
import requests
from bs4 import BeautifulSoup

homepage_url = 'https://www.linkedin.com'
login_url = 'https://www.linkedin.com/uas/login'
contacts_search_url = 'https://www.linkedin.com/connected/api/v2/contacts?start=40&count=10&fields=id%2Cname%2CfirstName%2ClastName%2Ccompany%2Ctitle%2Clocation%2Ctags%2Cemails%2Csources%2CdisplaySources%2CconnectionDate%2CsecureProfileImageUrl&sort=CREATED_DESC&_=1481999304007'

client = requests.Session()

homepage_html = client.get(homepage_url).content
soup = BeautifulSoup(homepage_html)
csrf_token = soup.find(id='loginCsrfParam-login')['value']

auth_payload = {
    'session_key' : email,
    'session_password' : password, 
    'loginCsrfParam' : csrf_token,
}

#print(csrf_token)

#r = client.post(login_url, data=auth_payload)
#r.text

client.post(login_url, data=auth_payload)
client.get(contacts_search_url)

Notes: it's quite possible there's another cookie involved in authenticating to the LinkedIn Contacts API, or perhaps some alternative method of authenticating.  The lidc cookie, and that combined with the CSRF token, didn't seem to have any effect.

It's also possible I just don't have the correct construction of the request, and that the data provided would work fine if asserted correctly.